In [1]:
from  langgraph.graph import StateGraph,END,START
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [2]:
## Define the state
class CrashState(TypedDict):
    """The state of the crash."""
    input: str
    step1: str
    step2: str
    step3: str
    

In [3]:
## Define Steps
def step_1(state: CrashState)-> CrashState:
    
    print("Step 1 Executed")
    
    return {'step1':"done","input":state['input']}

def step_2(state: CrashState) -> CrashState:
    print("Step 2 Hanging.... now manualyy interrupt from the notebook toolbar (STOP BUTTON)")
    time.sleep(10)
    return {'step2': 'Done'}

def step_3(state: CrashState) -> CrashState:
    print("Step 3 Executed")
    return {'step3': 'Done'}

In [5]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [12]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
Step 1 Executed
Step 2 Hanging.... now manualyy interrupt from the notebook toolbar (STOP BUTTON)
❌ Kernel manually interrupted (crash simulated).


In [13]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...
Step 2 Hanging.... now manualyy interrupt from the notebook toolbar (STOP BUTTON)
Step 3 Executed

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'Done', 'step3': 'Done'}


In [14]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'Done', 'step3': 'Done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f06ddf2-a4d1-6922-800b-db00c697c503'}}, metadata={'source': 'loop', 'writes': {'step_3': {'step3': 'Done'}}, 'step': 11, 'parents': {}, 'thread_id': 'thread-1'}, created_at='2025-07-31T07:22:50.331574+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f06ddf2-a4ce-6dad-800a-0f8975e4860e'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'Done', 'step3': 'Done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f06ddf2-a4ce-6dad-800a-0f8975e4860e'}}, metadata={'source': 'loop', 'writes': {'step_2': {'step2': 'Done'}}, 'step': 10, 'parents': {}, 'thread_id': 'thread-1'}, created_at='2025-07-31T07:22:50.330462+00:00', parent_config={'configurab